In [1]:
import pandas as pd
import numpy as np

import spacy
nlp = spacy.load('en_core_web_lg')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN, MiniBatchKMeans, KMeans, Birch, AffinityPropagation
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import wordcloud
import string

In [2]:
course = pd.read_csv('./data/archive_6/udemy_courses.csv')

In [3]:
course.head()

,Unnamed: 0,course_id,url,title,is_paid,instructors,image_480x270,published_title,headline,num_subscribers,...,num_published_practice_tests,has_closed_caption,created_at,instructional_level,published_at,objectives_summary,is_recently_published,last_update_date,content_info,category
0,0,655608,/course/productivity-and-time-management/,Productivity and Time Management for the Overw...,True,['Josh Paulsen'],https://img-b.udemycdn.com/course/480x270/6556...,productivity-and-time-management,A complete productivity and time management co...,130315,...,0,True,2015-10-30T15:09:33Z,All Levels,2015-11-10T22:18:22Z,['Take control of their mindset and energy lev...,False,2017-06-14,2 total hours,personal_development
1,1,591930,/course/become-a-superlearner-2-speed-reading-...,Become a SuperLearner® 2: Learn Speed Reading ...,True,"['Jonathan Levi', 'Lev Goldentouch', 'Anna Gol...",https://img-b.udemycdn.com/course/480x270/5919...,become-a-superlearner-2-speed-reading-memory-a...,The original course to learn faster & more eas...,201322,...,0,True,2015-08-24T15:22:20Z,All Levels,2015-12-03T18:33:51Z,['Learn to read roughly 3 times the speed of a...,False,2021-10-08,5.5 total hours,personal_development
2,2,1080376,/course/life-coaching-online-certification-cou...,Life Coaching Certificate Course (Beginner to ...,True,"['Kain Ramsay', 'The Academy of Modern Applied...",https://img-b.udemycdn.com/course/480x270/1080...,life-coaching-online-certification-course-life...,A comprehensive online training that delivers ...,145142,...,0,True,2017-01-17T22:38:10Z,All Levels,2017-03-11T03:58:33Z,['A time-proven life coaching methodology that...,False,2022-01-24,33 total hours,personal_development
3,3,2108272,/course/do-more-stress-less/,"Time Management Mastery: Do More, Stress Less",True,['Alexis Haselberger'],https://img-b.udemycdn.com/course/480x270/2108...,do-more-stress-less,"Take control of your time, your to-do list, an...",67441,...,1,True,2018-12-26T17:41:09Z,All Levels,2019-04-11T17:04:22Z,['Discover and apply time-management and produ...,False,2021-10-21,2 total hours,personal_development
4,4,1336872,/course/nlp-practitioner-neuro-linguistic-prog...,Neuro-Linguistic Programming (NLP) Practitione...,True,"['Kain Ramsay', 'The Academy of Modern Applied...",https://img-b.udemycdn.com/course/480x270/1336...,nlp-practitioner-neuro-linguistic-programming-...,Gain an in-depth understanding of Neuro-Lingui...,124804,...,0,True,2017-08-31T16:14:17Z,All Levels,2017-10-09T16:23:41Z,['The conscious use of language – Milton & Met...,False,2021-12-10,44 total hours,personal_development


In [4]:
# dropurl
course = course.drop(columns=['url', 'is_paid', 'image_480x270', 'num_subscribers', 'num_published_practice_tests', 'has_closed_caption', 'created_at', 'published_at', 'is_recently_published', 'last_update_date', 'content_info'])

In [5]:
course.head()

,Unnamed: 0,course_id,title,instructors,published_title,headline,avg_rating,avg_rating_recent,rating,num_reviews,num_published_lectures,instructional_level,objectives_summary,category
0,0,655608,Productivity and Time Management for the Overw...,['Josh Paulsen'],productivity-and-time-management,A complete productivity and time management co...,4.513310,4.499236,4.499236,47389,32,All Levels,['Take control of their mindset and energy lev...,personal_development
1,1,591930,Become a SuperLearner® 2: Learn Speed Reading ...,"['Jonathan Levi', 'Lev Goldentouch', 'Anna Gol...",become-a-superlearner-2-speed-reading-memory-a...,The original course to learn faster & more eas...,4.519956,4.525595,4.525595,34746,63,All Levels,['Learn to read roughly 3 times the speed of a...,personal_development
2,2,1080376,Life Coaching Certificate Course (Beginner to ...,"['Kain Ramsay', 'The Academy of Modern Applied...",life-coaching-online-certification-course-life...,A comprehensive online training that delivers ...,4.608515,4.617931,4.617931,34010,131,All Levels,['A time-proven life coaching methodology that...,personal_development
3,3,2108272,"Time Management Mastery: Do More, Stress Less",['Alexis Haselberger'],do-more-stress-less,"Take control of your time, your to-do list, an...",4.505920,4.514834,4.514834,27256,45,All Levels,['Discover and apply time-management and produ...,personal_development
4,4,1336872,Neuro-Linguistic Programming (NLP) Practitione...,"['Kain Ramsay', 'The Academy of Modern Applied...",nlp-practitioner-neuro-linguistic-programming-...,Gain an in-depth understanding of Neuro-Lingui...,4.576991,4.563306,4.563306,22499,167,All Levels,['The conscious use of language – Milton & Met...,personal_development


In [6]:
course = course.drop(columns=['avg_rating', 'avg_rating_recent', 'rating', 'num_reviews', 'num_published_lectures'])

In [7]:
course.head()

,Unnamed: 0,course_id,title,instructors,published_title,headline,instructional_level,objectives_summary,category
0,0,655608,Productivity and Time Management for the Overw...,['Josh Paulsen'],productivity-and-time-management,A complete productivity and time management co...,All Levels,['Take control of their mindset and energy lev...,personal_development
1,1,591930,Become a SuperLearner® 2: Learn Speed Reading ...,"['Jonathan Levi', 'Lev Goldentouch', 'Anna Gol...",become-a-superlearner-2-speed-reading-memory-a...,The original course to learn faster & more eas...,All Levels,['Learn to read roughly 3 times the speed of a...,personal_development
2,2,1080376,Life Coaching Certificate Course (Beginner to ...,"['Kain Ramsay', 'The Academy of Modern Applied...",life-coaching-online-certification-course-life...,A comprehensive online training that delivers ...,All Levels,['A time-proven life coaching methodology that...,personal_development
3,3,2108272,"Time Management Mastery: Do More, Stress Less",['Alexis Haselberger'],do-more-stress-less,"Take control of your time, your to-do list, an...",All Levels,['Discover and apply time-management and produ...,personal_development
4,4,1336872,Neuro-Linguistic Programming (NLP) Practitione...,"['Kain Ramsay', 'The Academy of Modern Applied...",nlp-practitioner-neuro-linguistic-programming-...,Gain an in-depth understanding of Neuro-Lingui...,All Levels,['The conscious use of language – Milton & Met...,personal_development


In [8]:
course.dropna(inplace=True)

In [10]:
course.shape

(111018, 9)

In [11]:
vec = TfidfVectorizer(stop_words='english', max_df=0.5, min_df=2)

In [14]:
features = vec.fit_transform(course["title"])

In [32]:
cls = KMeans(n_clusters=1000, n_init=20, max_iter=400, random_state=0)

In [33]:
cls.fit(features)

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_sparse'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_sparse'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


In [17]:
cls.predict(features)

array([  6, 304, 224, ..., 368,  69,  27])

In [25]:
def recommend_course(course_name):
    # tokenize name
    course_vector = vec.transform([course_name])
    course_cluster = cls.predict(course_vector)[0]
    return course[cls.labels_ == course_cluster].sample(n=5)

In [26]:
recommend_course('Machine Learning')

,Unnamed: 0,course_id,title,instructors,published_title,headline,instructional_level,objectives_summary,category
105637,105637,2092098,Machine Learning Intro for Python Developers,['Frank Anemaet'],machine-learning-intro-for-python-developers,Get started with Machine Learning Algorithms,Beginner Level,"['Machine Learning Basics with Python', 'How t...",development
106960,106960,2571224,Efficient Machine Learning,['Usama Albaghdady'],efficient-machine-learning,Become an Advanced Machine Learning Specialist...,Intermediate Level,"['Master Machine Learning', 'Performing Ideal ...",development
66704,66704,3994202,Learning Business Etiquette,['Sinan Cetiner'],learning-business-etiquette-q,Business etiquette is the glue that binds peop...,All Levels,"['Learn Telephone Etiquette', 'Learn Ethics an...",business
101872,101872,4248756,2022 Python and Machine Learning in Financial ...,['S.Emadedin Hashemi'],python-and-machine-learning-in-financial-analysis,Using Python and machine learning in financial...,All Levels,['You will be able to use the functions provid...,development
64351,64351,2283598,Machine Learning 101 with Scikit-learn and Sta...,['365 Careers'],machine-learning-101-with-scikit-learn-and-sta...,New to machine learning? This is the place to ...,All Levels,['You will gain confidence when working with 2...,business


In [31]:
recommend_course('react')

,Unnamed: 0,course_id,title,instructors,published_title,headline,instructional_level,objectives_summary,category
86687,86687,3957808,Производство. Бухгалтерский учёт,['Tatiana Sidorova'],ujjditsy,От проводок - к финансовой отчётности,Intermediate Level,['Производство. Бухгалтерский учёт'],finance
6388,6388,4224642,Meeting your Twin Flame in the 5th Dimension.,['Ioanna Tzanakaki'],meeting-your-twin-flame-in-the-5th-dimension,Re-discover your natural powers and enhance yo...,All Levels,['You will learn how to cultivate your ability...,personal_development
84624,84624,3051118,Kostenrechnung,['Thomas Wala'],kostenrechnung,Eine Einführung für Nicht-Betriebswirte,Beginner Level,['die Begriffe „Kosten“ und „Erlöse“ zu defini...,finance
22372,22372,3778896,WineMasters Class 1 - Wine course ranking high...,"['Andreas Larsson', 'Christy Canterbury MW']",winemastersclass1,"Wine MasterClass: Austrian Whites, California,...",Intermediate Level,"['Learn about all wine regions, grapes, the wi...",lifestyle
87168,87168,4344962,Capital Gain (Indian Income tax ) (ಕನ್ನಡದಲ್ಲಿ ...,['Jnanesha AT'],capital-gain-indian-income-tax-kannada-b_com-b...,"B Com, BBA ಮತ್ತು Taxation ವಿದ್ಯಾರ್ಥಿಗಳಿಗಾಗಿ ಆದ...",Beginner Level,['The student will learn Basic Knowledge about...,finance
